In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/self_supervised2/data/archive.zip"


In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import keras
import sys
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
import tensorflow_datasets as tfds
import os
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow_addons as tfa

In [33]:
input_shape = (96,96,3)
dataset_name = "stl10"
AUTOTUNE = tf.data.AUTOTUNE
shuffle_buffer = 5000

In [44]:


def data_stl(size = 32):

    unlabeled_batch_size = size
    labeled_batch_size = size
    batch_size = size

    train_dataset = (
        tfds.load(dataset_name, split="train", as_supervised=True, shuffle_files=True)

    )
    test_dataset = (
        tfds.load(dataset_name, split="test", as_supervised=True)
    )

    X_train, y_train = tuple(zip(*train_dataset))

    X_test, y_test = tuple(zip(*test_dataset))

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    y_train2 = np.zeros((y_train.size, y_train.max()+1), dtype=int)
    y_train2[np.arange(y_train.size),y_train] = 1

    y_test2 = np.zeros((y_test.size, y_test.max()+1), dtype=int)
    y_test2[np.arange(y_test.size),y_test] = 1



    train_data = tf.data.Dataset.from_tensor_slices((tf.constant(X_train),
                                                tf.constant(y_train2)))


    train_data = train_data.shuffle(buffer_size=len(X_train))


    train_data = train_data.batch(batch_size , drop_remainder=True)



    test_data = tf.data.Dataset.from_tensor_slices((tf.constant(X_test),
                                                tf.constant(y_test2)))


    test_data = test_data.shuffle(buffer_size=len(X_test))


    test_data = test_data.batch(batch_size , drop_remainder=True)



    return train_data, test_data


In [48]:
#Tiny Imagenet

def process_image(image_path , IMG_SIZE = 96):

  image = tf.io.read_file(image_path)

  image = tf.image.decode_jpeg(image, channels=3)

  image = tf.image.convert_image_dtype(image, tf.float32)

  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])

  return image


def get_image_label(image_path, label):

  image = process_image(image_path)
  return image, label


def data_imagenet(batch_size=32):


    base  = "/content/tiny-imagenet-200/train"

    files = []
    labels = []
    i = 0
    for p in sorted(os.listdir(base)):

        base2 =  os.path.join(base,p,"images")

        t = []
        t1 = []
        for p2 in sorted(os.listdir(base2)):

            t.append(os.path.join(base2,p2))
            t1.append(i)

        files.append(t)
        labels.append(t1)
        i+=1



    files = np.array(files)
    labels = np.array(labels)


    files = files.flatten()
    labels = labels.flatten()



    print(files.shape)
    print(labels.shape)


    X_train, X_val_, y_train, y_val_ = train_test_split(files,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=42)


    print()
    print()

    print("X_train ", X_train.shape)
    print("X_val_ ", X_val_.shape)
    print("y_train ", y_train.shape)
    print("y_val_ ", y_val_.shape)

    print()
    print()




    X_val, X_test, y_val, y_test = train_test_split(X_val_,
                                                    y_val_,
                                                    test_size=0.1,
                                                    random_state=42)



    print()
    print()

    print("X_val ", X_val.shape)
    print("X_test ", X_test.shape)
    print("y_val ", y_val.shape)
    print("y_test ", y_test.shape)

    print()
    print()


    print()
    print()

    print("X_train ", X_train.shape)
    print("y_train ", y_train.shape)
    print("X_val ", X_val.shape)
    print("y_val ", y_val.shape)


    print()
    print()


    AUTO = tf.data.AUTOTUNE



    y_val = tf.constant(y_val)
    depth = int(tf.reduce_max(y_val))+1
    y_val = tf.one_hot(y_val , depth)


    y_test = tf.constant(y_test)
    y_test = tf.one_hot(y_test , depth)





    val_data = tf.data.Dataset.from_tensor_slices((tf.constant(X_val),
                                            y_val))


    val_data = val_data.shuffle(buffer_size=len(X_val))


    val_data = val_data.map(get_image_label).batch(batch_size , drop_remainder=True)









    test_data = tf.data.Dataset.from_tensor_slices((tf.constant(X_test),
                                            y_test))


    test_data = test_data.shuffle(buffer_size=len(X_test))


    test_data = test_data.map(get_image_label).batch(batch_size , drop_remainder=True)




    val_data = val_data.prefetch(buffer_size=AUTO)

    test_data = test_data.prefetch(buffer_size=AUTO)


    return (val_data , test_data )



In [ ]:
#Cifar-10


def crop(image,mask , CROP_TO = 96 ):
    # With random crops we also apply horizontal flipping.
    # image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, size = [CROP_TO, CROP_TO])
    return image, mask



def data_cifar(size = 32 , CROP_TO = 96,name = "cifar10"):

    if(name == "cifar10"):

        (Xc_train, y_train), (Xc_test, y_test) = tf.keras.datasets.cifar10.load_data()

    else:

        (Xc_train, y_train), (Xc_test, y_test) = tf.keras.datasets.cifar100.load_data()

    y_train = np.squeeze(y_train, axis=1)

    y_test = np.squeeze(y_test, axis=1)


    print(" Xc_train   yf_train " , Xc_train.shape , y_train.shape)
    print(" Xc_test yv_test" ,  Xc_test.shape ,  y_test.shape)

    print()
    print()
    print()
    print()




    SEED = 42



    AUTO = tf.data.AUTOTUNE



    y_train = tf.constant(y_train)
    depth = int(tf.reduce_max(y_train))+1
    y_train = tf.one_hot(y_train , depth)



    y_test = tf.constant(y_test)
    y_test = tf.one_hot(y_test , depth)



    train_ds = tf.data.Dataset.from_tensor_slices((tf.constant(Xc_train), y_train))

    test_ds = tf.data.Dataset.from_tensor_slices((tf.constant(Xc_test), y_test))



    train_ds = train_ds.map(crop).batch(size).prefetch(AUTO)


    test_ds = test_ds.map(crop).batch(size).prefetch(AUTO)



    print("size -- " , size)

    print("train_ds --  " , train_ds)


    print("test_ds-- " , test_ds)

    return (train_ds , test_ds )


In [37]:

class RandomResizedCrop(layers.Layer):
    def __init__(self, scale, ratio):
        super(RandomResizedCrop, self).__init__()
        self.scale = scale
        self.log_ratio = (tf.math.log(ratio[0]), tf.math.log(ratio[1]))

    def call(self, images):
        batch_size = tf.shape(images)[0]
        height = tf.shape(images)[1]
        PROJECT_DIM = tf.shape(images)[2]

        random_scales = tf.random.uniform((batch_size,), self.scale[0], self.scale[1])
        random_ratios = tf.exp(
            tf.random.uniform((batch_size,), self.log_ratio[0], self.log_ratio[1])
        )

        new_heights = tf.clip_by_value(tf.sqrt(random_scales / random_ratios), 0, 1)
        new_PROJECT_DIMs = tf.clip_by_value(tf.sqrt(random_scales * random_ratios), 0, 1)
        height_offsets = tf.random.uniform((batch_size,), 0, 1 - new_heights)
        PROJECT_DIM_offsets = tf.random.uniform((batch_size,), 0, 1 - new_PROJECT_DIMs)

        bounding_boxes = tf.stack(
            [
                height_offsets,
                PROJECT_DIM_offsets,
                height_offsets + new_heights,
                PROJECT_DIM_offsets + new_PROJECT_DIMs,
            ],
            axis=1,
        )
        images = tf.image.crop_and_resize(
            images, bounding_boxes, tf.range(batch_size), (height, PROJECT_DIM)
        )
        return images



class RandomBrightness(layers.Layer):
    def __init__(self, brightness):
        super(RandomBrightness, self).__init__()
        self.brightness = brightness

    def blend(self, images_1, images_2, ratios):
        return tf.clip_by_value(ratios * images_1 + (1.0 - ratios) * images_2, 0, 1)

    def random_brightness(self, images):
        # random interpolation/extrapolation between the image and darkness
        return self.blend(
            images,
            0,
            tf.random.uniform(
                (tf.shape(images)[0], 1, 1, 1), 1 - self.brightness, 1 + self.brightness
            ),
        )

    def call(self, images):
        images = self.random_brightness(images)
        return images

def augmenter(brightness, name, scale):
    return keras.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Rescaling(1 / 255),
            layers.RandomFlip("horizontal"),
            RandomResizedCrop(scale=scale, ratio=(3 / 4, 4 / 3)),
            RandomBrightness(brightness=brightness),
        ],
        name=name,
    )


def augmenter2( brightness, name, scale):
    return keras.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Rescaling(1 / 255),
            layers.RandomFlip("horizontal"),
            layers.RandomZoom(0.2),
            layers.RandomRotation(0.3),
            layers.RandomContrast(0.2),
            layers.RandomHeight(0.2),

            RandomResizedCrop(scale=scale, ratio=(3 / 4, 4 / 3)),
            RandomBrightness(brightness=brightness),
        ],
        name=name,
    )



In [38]:

def evalute_method(model, val, test,n , l =  0.001,l1 =  0.001,epochs = 50 ,size = 32, input_shape = (96,96,3),r = False):






    print()
    print()
    print()
    print("______________________________")
    print("Hyper parameter...")
    print(" n = ", n)
    print(" l = ",  l)
    print(" l1 = ", l1)
    print(" epochs = ", epochs)
    print(" size = ", size)
    print(" input_shape = ", input_shape )
    print()
    print()
    print()
    print()


    finetuning_model = keras.Sequential(
        [
            layers.Input(shape=input_shape),
            augmenter( brightness = 0.2, name =  'contrastive_augmenter', scale =  (0.5, 1.0)),
            model,
            layers.Dense(n  ,activation = "softmax"),
        ],
        name="finetuning_model",
    )




    finetuning_model.compile(
        optimizer=keras.optimizers.Adam(l),
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.CategoricalAccuracy(name="Accuracy") ,
                tfa.metrics.F1Score(num_classes = n ,  name='F1_score'),
                    tf.keras.metrics.Precision(name = "Precision"),
                    tfa.metrics.RSquare(name = "R2 score"),
                    tf.keras.metrics.Recall(name = "Recall"),
                    tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='Top 5 Accuracy'),
                    tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='Top 3 Accuracy')

                    ],
    )

    history = finetuning_model.fit(
        val, epochs=epochs, validation_data=test
    )



    # res = finetuning_model.evaluate(test)

    df = history.history

    avg_train_F1 = [ sum(i)/len(i) for i in  df["F1_score"]]
    avg_val_F1  = [ sum(i)/len(i) for i in  df["val_F1_score"]]

    df["F1_score"] = avg_train_F1
    df["val_F1_score"] = avg_val_F1

    if(r):

        return finetuning_model, df

    return df


In [39]:
def get_encoder2():
    base_model = tf.keras.applications.ResNet50(include_top=False,
        weights="imagenet", input_shape=input_shape)
    base_model.trainable = True

    inputs = tf.keras.layers.Input(input_shape)
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(2048, activation='relu', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    z = tf.keras.layers.Dense(2048)(x)

    f = tf.keras.Model(inputs, z)

    return f



In [ ]:
train_imagenet , test_imagenet = data_imagenet()
train_stl , test_stl  = data_stl()

In [ ]:
model_ =  get_encoder2()

model_.load_weights('/content/drive/MyDrive/self_supervised2/model_pNNCLR_WOB_mark-x_imgnet.h5')

model_ = tf.keras.Model(model_.input, model_.layers[2].output)

model_.trainable = False

df1 = evalute_method(model_,train_stl , test_stl , n = 10 )